## Otimização de hiperparâmetros



## Introdução



Escolheu-se "árvores aleatórias" como algoritmo para otimizar três hiperparâmetros utilizando-se de redes neurais para achar o melhor conjunto de hiperparâmetros.


## Objetivo



**Objetivo**: use algoritmos genéticos para encontrar um bom conjunto de hiperparâmetros em um experimento de aprendizado de máquina. Escolha um algoritmo que tenha pelo menos 3 hiperparâmetros para serem otimizados.



## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
import random
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from operator import itemgetter

## Códigos e discussão



-   Use células de código para o código.

-   Use células de texto para a discussão.

-   A discussão não deve ser feita em comentários dentro das células de código. Toda discussão deve acontecer após o resultado sendo discutido foi apresentado. Exemplo: não discuta um gráfico antes de apresentá-lo.



In [2]:
######################################
# CÓDIGO INCOMPLETO, EM FASE FINAL DE TESTES
######################################
#O QUE PRECISA SER FEITO:
#resolver **dicio -- **args, **kwargs and * operator:
#https://stackoverflow.com/questions/36901/what-does-double-star-asterisk-and-star-asterisk-do-for-parameters

#docstrings + comentarios

######################################

O notebook GA.08 é um dos experimentos da "Lista de experimentos - algoritmos genéticos" e aborda o problema de otimização de hiperparâmetros. O código atualmente não está completo, de modo que está em fase final de testes (por causa da aplicação do **dicio vs. **args e o operador asterisco. Como código está, ele define um conjunto de hiperparâmetros que se quer investigar uma otimização de MSE. Uma nota interessante é que a função objetivo é baseada no valor do MSE do individuo (fitness = MSE), de modo em que caso este valor for superior ao baseline (valor de MSE com hiperparâmetros padrão), deve-se aplicar uma punição de 1e6 no fitness (fitness = 1e6 + MSE), aplocado a uma função de minimização.

Além disso, foi criado uma função de mutação (mutacao_tendencia_as_cegas) que pega 3 individuos com menor MSE, escolhe um deles aleatoriamente e escolhe um dos genes automaticamente. Após isso, substitui-se um dos genes do individuo por aquele selecionado, proveniente de um dos 3.

Outra funcao particularmente interessante é a função de cruzamento (reproducao_combinacao_hiperparametros), que gera 3 individuos pela combinacao aleatoria de genes de uma "mae", "pai" e "pae", naturalmente respetando a ordem de hiperparametros.

No código, ainda faltam as docstrings e comentários, além da aplicação da "sat asterisk" e "double star asterisk". Quer-se investigar sobre qual seria a melhor ferramenta para a aplciação. Há, ainda, alguns problemas. Aparentemente a aplicação de conjuntos para armazenar os hiperparâmetros e seus valores de MSE.
Nota: as funções não estão definidas em funçoes.py pois o código ainda está em fase (final) de testes

In [4]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 9
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
CHANCE_MUTACAO_2 = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3
NUM_GERACOES = 5
NUM_GENES = 5
CONJUNTOS_HIPERPARAMETROS = 3
CORTE = 3

# relacionadas ao problema a ser resolvido - dataset
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 1024
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]
HIPER_RANGE_1=20
HIPER_RANGE_2=10


# relacionadas ao problema a ser resolvido - hiperparametros
#irei variar "NUM_FOLHAS, NUM_PROFUNDIDADE, MINIMO_FOLHAS"
#deixei aqui o resto para demonstrar o que se poderia variar. o resto está nos valores padrão
#NUM_FOLHAS = 50 #max_leaf_nodes
#NUM_PROFUNDIDADE = 70 #max_depth
#CRITERIO = 'absolute_error'
SPLITTER = "best"
#MINIMO_SPLIT = 2 #min_samples_split
MINIMO_FOLHAS = 1
PESO_FOLHAS = 0.0
MAXIMO_FEATURES = None
IMPUREZA_MINIMO = 0.0
COMPLEXO_ALPHA = 0.0


In [5]:
df = sns.load_dataset(DATASET_NAME) #criação do dataset
#df

In [6]:
tamanho = 0.25 # Fração de dados escolhida para treino e teste
seed = SEMENTE_ALEATORIA

i = df.index
i_treino, i_teste = train_test_split(i, test_size=tamanho, random_state = seed)

df_treino = df.loc[i_treino]
df_teste = df.loc[i_teste]

In [7]:
X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values
X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values

In [8]:
dicio = {
    'a':10,
    'b':11,
    'c':14,
}
#dicio_2 = (a = 10, b = 11, c = 14)
def func(a,b,c, **kwargs):
    print(a)

In [9]:
func(**dicio)

10


In [10]:
def hiperparametros_modelo(hiper_range_1,hiper_range_2):

    hiperparametros = {
    'max_depth':([max_depth for max_depth in range(1,hiper_range_1)]), #max_depth
    'criterion':('squared_error', 'friedman_mse', 'absolute_error', 'poisson'), #criterion
    'min_samples_split':([min_samples_split for min_samples_split in range(2,hiper_range_2)]) #min_samples_split
    }
    
    return hiperparametros
    

In [11]:
def individuo_hiperparametros(hiper_range_1,hiper_range_2):
    '''Gera um valor aleatório para cada hiperparâmetro em uma dada faixa.
    
    Returns:
        Um valor válido para os 3 hiparparâmetros variados
        '''
    
    individuo = []
    hiperparametros = hiperparametros_modelo(hiper_range_1,hiper_range_2)
    for i in (range(0,len(hiperparametros))):
        gene = random.sample(list(hiperparametros.values())[i], k=1)
        individuo.append(gene[0]) #retira da lista gerada por random.sample()
        
    return individuo
        

In [12]:
#individuo_hiperparametros(20,10)

In [13]:
def populacao_inicial_hiperparametros(tamanho, hiper_range_1,hiper_range_2):

    populacao = []
    while len(populacao) != tamanho:
        for _ in range(tamanho):
            populacao.append(individuo_hiperparametros(hiper_range_1,hiper_range_2)) #cria uma lista de listas como populacao
        #populacao_2 = set(populacao)
    return populacao


In [14]:
#pop = populacao_inicial_hiperparametros(10,20,10)
#print(pop)

In [15]:
def selecao_torneio_min_hiperparametros(populacao, fitness, tamanho_torneio=3): #implementada em funcoes.py
    """Faz a seleção de uma população usando torneio.

    Nota: da forma que está implementada, só funciona em problemas de
    minimização.

    Args:
      populacao: população do problema
      fitness: lista com os valores de fitness dos individuos da população
      tamanho_torneio: quantidade de invidiuos que batalham entre si

    Returns:
      Individuos selecionados. Lista com os individuos selecionados com mesmo
      tamanho do argumento `populacao`.
    """
    selecionados = []
    fitness_selecionados = []
    minimo_fitness_list = []
    #index = -1

    # criamos essa variável para associar cada individuo com seu valor de fitness
    par_populacao_fitness = list(zip(populacao, fitness))

    # vamos fazer len(populacao) torneios! Que comecem os jogos!
    for _ in range(len(populacao)):
        combatentes = random.sample(par_populacao_fitness, tamanho_torneio)

        # é assim que se escreve infinito em python
        minimo_fitness = float("inf")

        for par_individuo_fitness in combatentes:
            individuo = par_individuo_fitness[0]
            fit = par_individuo_fitness[1]

            # queremos o individuo de menor fitness
            if fit < minimo_fitness:
                selecionado = individuo
                minimo_fitness = fit
                selecionados.append(selecionado)
                minimo_fitness_list.append(minimo_fitness)
               

    par_selecionados_fitness = list(zip(selecionados, minimo_fitness_list))
    par_selecionados_fitness_sorted = sorted(par_populacao_fitness, key=itemgetter(-1))
    selecionados, fitness_selecionados = zip(*par_selecionados_fitness_sorted)

    return list(selecionados), list(fitness_selecionados)


In [16]:
fitness_pop = [2108462.143928877,
 2313900.0381905823,
 2179001.386472449,
 2293625.6872774656,
 7541357.63714644,
 7541357.63714644,
 2178735.6442930433,
 2466910.6050827727,
 2135296.6684523607,
 2357057.1922043012]

In [17]:
#selecao_torneio_min_hiperparametros(pop, fitness_pop, tamanho_torneio=3) #implementada em funcoes.py

In [18]:
def funcao_objetivo_standard():

    indices = df.index
    indices_treino, indices_teste = train_test_split(
        indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
    )

    df_treino = df.loc[indices_treino]
    df_teste = df.loc[indices_teste]

    # observe que usamos o .values aqui pois queremos apenas os valores
    X_treino = df_treino.reindex(FEATURES, axis=1).values
    y_treino = df_treino.reindex(TARGET, axis=1).values
    X_teste = df_teste.reindex(FEATURES, axis=1).values
    y_teste = df_teste.reindex(TARGET, axis=1).values
    
    modelo_dt = DecisionTreeRegressor(random_state=SEMENTE_ALEATORIA)

    # treina o modelo
    modelo_dt.fit(X_treino, y_treino)
    
    y_verdadeiro = y_teste
    y_previsao = modelo_dt.predict(X_teste)

    MSE_standard = mean_squared_error(y_verdadeiro, y_previsao, squared=True) #calcular MSE sem alterar nenhum parâmetro
    
    return MSE_standard

In [19]:
#MSE_standard = funcao_objetivo_standard() #remove

In [20]:
#print(type(pop))

In [21]:
def funcao_objetivo_hiperparametros(individuo,MSE_standard,punishment=1e6):
    
    
    [num_profundidade, qualidade_criterio, split_folhas] = individuo
    
    modelo_dt = DecisionTreeRegressor(
        criterion=qualidade_criterio,
        max_depth=num_profundidade,
        min_samples_split=split_folhas,
        random_state=1024,
    )

    indices = df.index
    indices_treino, indices_teste = train_test_split(
        indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
    )

    df_treino = df.loc[indices_treino]
    df_teste = df.loc[indices_teste]

    # observe que usamos o .values aqui pois queremos apenas os valores
    X_treino = df_treino.reindex(FEATURES, axis=1).values
    y_treino = df_treino.reindex(TARGET, axis=1).values
    X_teste = df_teste.reindex(FEATURES, axis=1).values
    y_teste = df_teste.reindex(TARGET, axis=1).values
    
    #modelo_dt = DecisionTreeRegressor(random_state=SEMENTE_ALEATORIA)

    # treina o modelo
    modelo_dt.fit(X_treino, y_treino)
    
    y_verdadeiro = y_teste
    y_previsao = modelo_dt.predict(X_teste)

    MSE = mean_squared_error(y_verdadeiro, y_previsao, squared=True)
    
    fitness = 0
    
    if MSE > MSE_standard:
        fitness = MSE + punishment
    else:
        fitness = MSE
    
    return fitness

In [22]:
#funcao_objetivo_hiperparametros([9, 'friedman_mse', 6],MSE_standard)

In [23]:
def funcao_objetivo_pop_hiperparametros(populacao,MSE_standard):

    #print(populacao)
    fitness_pop = []
    for individuo in populacao:
        populacao_MSE_fit = funcao_objetivo_hiperparametros(individuo,MSE_standard)
        fitness_pop.append(populacao_MSE_fit)
        
    return fitness_pop

In [24]:
#funcao_objetivo_pop_hiperparametros(pop,MSE_standard)

In [25]:
def mutacao_tendencia_as_cegas(individuo, populacao, corte=3):
    
    print(individuo)
    #lista_index = [i for i in range(0, (len(individuo)))]
    range_index_individuo = range(0,len(individuo))
    
    melhores_individuos = populacao[:corte] #pegar os melhores individuos baseado no sort pelo fitness
    
    individuo_melhor = (random.choice(melhores_individuos)) #escolher um melhor individuo baseado no corte
    print('invidivuo_melhor: ',individuo_melhor)

    numero_mutacoes = random.randint(1, len(individuo)) #escolher o número de mutações que irá ocorrer
    print(numero_mutacoes)
    mutate_random_index = random.sample(range_index_individuo, numero_mutacoes) #escolher o index dessas mutações
    print(mutate_random_index)
    
    if individuo_melhor == individuo: #se pegou si mesmo, não faz sentido mutar
        return individuo
    else:
        for gene_index in mutate_random_index:
            individuo[gene_index] = individuo_melhor[gene_index] #substitui os index do individuo pelo individuo_melhor
        
    return individuo #individuo mutado

In [26]:
#mutacao_tendencia_as_cegas([2, 'absolute_error', 9],pop)

In [27]:
individuo = [1,2,3,4]
numero_mutacoes = 2
mutate_random_index = random.sample((range(0,len(individuo))), numero_mutacoes)
print(mutate_random_index)

[2, 1]


In [28]:
def mutacao_hiperparametros(individuo,hiper_range_1,hiper_range_2):
    hiperparametros_possiveis = hiperparametros_modelo(hiper_range_1,hiper_range_2)
    mutate_random_hp = random.choice(range(0,len(hiperparametros_possiveis)))
    #print(mutate_random_hp)
    mutate_random_hp_parameter = random.choice((list(hiperparametros_possiveis.values())[mutate_random_hp]))
    #print(mutate_random_hp_parameter)
    
    individuo[mutate_random_hp] = mutate_random_hp_parameter
    
    return individuo

In [29]:
mutacao_hiperparametros([17, 'friedman_mse', 7],20,10)

[6, 'friedman_mse', 7]

In [30]:
def reproducao_combinacao_hiperparametros(pai, mae, pae,numero_de_proles=3):
    
    #considere individuo e fit acoplado
    
    proles = []
    genes = []
    
    progenitores = [pai, mae, pae] #desacoplar
    lista_index = list(range(0,len(pai))) #indexes devem ser fixos para que a ordem não varie
    
    for _ in range(0,numero_de_proles):
        random.shuffle(progenitores) #lista de index aleatório
        #random.shuffle(lista_index)
    
        for index, parente in zip(lista_index, progenitores):
            #print(parente[index])
            genes.append(parente[index]) #recebe um gene aleatório de cada progenitor
            #print(genes)
            if len(genes) == len(pai):
                proles.append(genes)
                #print(individuo)
                genes = []
        
    return proles

In [31]:
reproducao_combinacao_hiperparametros([107, 'square', 7], [144, 'circle', 6], [9, 'dot', 222])

[[107, 'dot', 6], [144, 'dot', 7], [9, 'square', 6]]

In [32]:
p1,p2,p3 = reproducao_combinacao_hiperparametros([107, 'square', 7], [144, 'circle', 6], [9, 'dot', 222])
print(p1,p2,p3)

[144, 'square', 222] [144, 'square', 222] [9, 'square', 6]


In [33]:
lst = [0,1,2]
list(range(0,len(lst)))

[0, 1, 2]

In [34]:
#tendencia_as_cegas(individuo,pop,fitness,3)

In [35]:
#print(pop)
#ndividuo = [14, 'absolute_error', 7]

In [36]:
#lst = [1,2,3,4,5]
#print(lst[:3])

In [37]:
#individuo = pop[0]

In [38]:
#pop = populacao_inicial_hiperparametros(10,20,10)
#fitness = funcao_objetivo_pop_hiperparametros(pop)
#individuo = pop[3]

In [39]:
#pop = [[1, 'absolute_error', 5], [9, 'squared_error', 4]]

In [40]:
#funcao_objetivo_pop_hiperparametros(pop)

In [41]:
# funções locais

def cria_populacao_inicial(tamanho, hiper_range_1,hiper_range_2):
    return populacao_inicial_hiperparametros(tamanho, hiper_range_1,hiper_range_2)

def funcao_objetivo_pop(populacao,MSE_standard):
    return funcao_objetivo_pop_hiperparametros(populacao,MSE_standard)

def funcao_mutacao(individuo,hiper_range_1,hiper_range_2):
    return mutacao_hiperparametros(individuo,hiper_range_1,hiper_range_2)

def funcao_mutacao_2(individuo, populacao, corte=3):
    return mutacao_tendencia_as_cegas(individuo, populacao, corte=3)

def funcao_selecao(populacao, fitness, tamanho_torneio=3):
    return  selecao_torneio_min_hiperparametros(populacao, fitness, tamanho_torneio=3)

def funcao_cruzamento(pai,mae,pae):
    return reproducao_combinacao_hiperparametros(pai, mae, pae,numero_de_proles=3)

In [42]:
MSE_standard = funcao_objetivo_standard()
populacao = cria_populacao_inicial(TAMANHO_POP, HIPER_RANGE_1,HIPER_RANGE_2)
set_hiperparametros = set()
set_fitness = set()

while len(set_hiperparametros) != CONJUNTOS_HIPERPARAMETROS:
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao, MSE_standard)
    populacao, fitness_populacao = funcao_selecao(populacao, fitness)
    #print(populacao,type(fitness_populacao),fitness_populacao, type(fitness_populacao))
    
    # Mutação_2
    for k in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO_2:
            individuo = populacao[k]
            populacao[k] = funcao_mutacao_2(individuo,populacao,CORTE)    
    
    # Cruzamento
    pais = populacao[0::3]
    print(pais)
    maes = populacao[1::3]
    print(maes)
    paes = populacao[2::3]
    print(paes)
    
    #pais, fitness_pais = populacao[0::2], fitness_populacao[0::2]
    #maes, fitness_maes = populacao[1::2], fitness_populacao[1::2]
    #paes, fitness_paes = populacao[2::2], fitness_populacao[2::2]
    
    
    contador = 0
    
    for pai, mae, pae in zip(pais, maes, paes):
        if random.random() <= CHANCE_CRUZAMENTO: #or pai, mae, pae == pais[0], maes[0], paes[0]:
            filho_1, filho_2, filho_3 = funcao_cruzamento(pai, mae, pae)
            print(filho_1,filho_2,filho_3)
            print(populacao)
            populacao[contador] = filho_1
            print(contador)
            populacao[contador + 1] = filho_2
            print(contador)
            populacao[contador + 2] = filho_3
            print(contador)
            print(populacao)
        
        contador = contador + 3   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo,HIPER_RANGE_1,HIPER_RANGE_2)           
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao, MSE_standard)
    #print(fitness)
    menor_fitness = min(fitness)
    #print(menor_fitness)
    posicao = fitness.index(menor_fitness)
    melhor_individuo_ja_visto = (populacao[posicao]) #list is unhasheable
    #print(melhor_individuo_ja_visto)
    set_hiperparametros.add(tuple(melhor_individuo_ja_visto))
    #print(set_hiperparametros)
    set_fitness.add(tuple([menor_fitness]))
    #print(set_fitness)

print()
print(f'Aqui estão {CONJUNTOS_HIPERPARAMETROS} conjuntos de hiperparametros decentes:')
print(set_hiperparametros, set_fitness)

[16, 'absolute_error', 5]
invidivuo_melhor:  [8, 'friedman_mse', 9]
1
[0]
[[8, 'friedman_mse', 9], [10, 'friedman_mse', 8], [8, 'absolute_error', 5]]
[[6, 'squared_error', 8], [3, 'poisson', 9], [2, 'friedman_mse', 3]]
[[5, 'squared_error', 8], [16, 'poisson', 8], [18, 'squared_error', 2]]
[6, 'squared_error', 9] [6, 'friedman_mse', 8] [5, 'squared_error', 9]
[[8, 'friedman_mse', 9], [6, 'squared_error', 8], [5, 'squared_error', 8], [10, 'friedman_mse', 8], [3, 'poisson', 9], [16, 'poisson', 8], [8, 'absolute_error', 5], [2, 'friedman_mse', 3], [18, 'squared_error', 2]]
0
0
0
[[6, 'squared_error', 9], [6, 'friedman_mse', 8], [5, 'squared_error', 9], [10, 'friedman_mse', 8], [3, 'poisson', 9], [16, 'poisson', 8], [8, 'absolute_error', 5], [2, 'friedman_mse', 3], [18, 'squared_error', 2]]
[3, 'friedman_mse', 8] [10, 'poisson', 9] [16, 'friedman_mse', 9]
[[6, 'squared_error', 9], [6, 'friedman_mse', 8], [5, 'squared_error', 9], [10, 'friedman_mse', 8], [3, 'poisson', 9], [16, 'poisson', 8

## Conclusão



Delete este texto e escreva sua conclusão.



## Referências consultadas



1.  Delete este texto e inclua suas referências ordenadas numericamente. Se for referenciar no notebook, use o número entre colchetes (exemplo: para citar essa referência aqui escreva &ldquo;[1]&rdquo; sem as áspas).

2.  Cada item deve ser numerado. Siga o padrão apresentado.

3.  Caso não tenha nenhuma referência consultada, delete esta seção e o texto contido nela!



## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.



In [ ]:
dicio = {
    'a':10,
    'b':11,
    'c':14,
}
#dicio_2 = (a = 10, b = 11, c = 14)
def func(a,b,c, **kwargs):
    print(a)

In [43]:
print(tuple([[1],['a'],[2]]))

([1], ['a'], [2])


In [44]:
dicio = {
    'a':10,
    'b':11,
    'c':14,
}
#dicio_2 = (a = 10, b = 11, c = 14)
def func(a,b,c):
    print(a)

In [45]:
def tendencia_as_cegas(individuo, populacao, fitness_populacao, corte=3):
    
    print(individuo)
    lista_index = [i for i in range(0, (len(individuo)))]
    
    par_populacao_fitness = list(zip(populacao, fitness_populacao))
    par_populacao_fitness_sorted = sorted(par_populacao_fitness, key=itemgetter(-1))
    
    melhores_individuos_zip = par_populacao_fitness_sorted[:corte]
    
    individuo_melhor = random.choice(melhores_individuos_zip)[0]
    
    print('individuo',individuo_melhor)
    numero_mutacoes = random.randint(1, len(individuo))
    print(numero_mutacoes)
    mutate_random_index = random.sample(lista_index, numero_mutacoes)
    print(mutate_random_index)
    
    if individuo_melhor == individuo or numero_mutacoes == len(individuo):
        return individuo
    else:
        for gene_index in mutate_random_index:
            individuo[gene_index] = individuo_melhor[gene_index]
        
    return individuo

In [46]:
def gene_hiperparametros(max_depth_range,min_samples_split_range):
    '''Gera um valor aleatório para cada hiperparâmetro em uma dada faixa.
    
    Returns:
        Um valor válido para os 3 hiparparâmetros variados
        '''

    individuo = []
    hiperparametros = [
    [max_depth for max_depth in range(1,max_depth_range)], #max_depth
    ["gini","entropy","log_loss"], #criterion
    [min_samples_split for min_samples_split in range(2,min_samples_split_range)]
                        ]
    
    for i in (range(0,len(hiperparametros))):
        gene = random.sample(hiperparametros[i], k=1)
        individuo.append(gene)
        
    return individuo
        

In [47]:
def mutacao_hp(individuo,hiper_range_1,hiper_range_2):
    mutacao_individual = 0.25
    hiperparametros_possiveis = hiperparametros_modelo(hiper_range_1,hiper_range_2)
    for i in range(0,len(individuo)):
        if random.random() <= mutacao_individual:
            #mutate_random_hp = random.choice(range(0,len(hiperparametros_possiveis)))
            mutate_random_hp_parameter = random.choice((list(hiperparametros_possiveis.values())[i]))
    
            individuo[i] = mutate_random_hp_parameter
    
    return individuo

In [48]:
def funcao_objetivo_pop_hiperparametros(populacao):

    #print(populacao)
    fitness_pop = []
    high_fit =[] #fit superior a baseline
    low_fit = [] #fit inferior a baseline
    for individuo in populacao:
        populacao_MSE_fit = funcao_objetivo_hiperparametros(individuo)
        individuo = individuo.extend(populacao_MSE_fit)
    
    for individuo_MSE_fit in populacao_MSE_fit:
        if individuo_MSE_fit[-1] != individuo_MSE_fit[-2]: # index([-2,-1]) = [MSE, fitness]
            high_fit.append(individuo_MSE_fit)
            
        else:
            low_fit.append(individuo_MSE_fit)
            
    low_fit_sorted = sorted(low_fit, key=itemgetter(-1)) #itemgetter muito util para sort() em listas de listas
    for low_fit_individuo in low_fit_sorted:
        print(low_fit_sorted)
        low_fit_individuo[1] = low_fit.index(low_fit_individuo)*500
        
        
    return low_fit + high_fit
#ideia: detectar aqueles com o fit menor do que o standard, organiza-los em ordem crescente, de modo que index*500 == fitness